In [ ]:
# %load main.py
import math
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tqdm import tqdm


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    # Load the saved model
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)

    # Get the relevant layers for constructing the skip-layers out of the graph
    graph       = tf.get_default_graph()
    input_image = graph.get_tensor_by_name('image_input:0')
    keep_prob   = graph.get_tensor_by_name('keep_prob:0')
    layer3_out  = graph.get_tensor_by_name('layer3_out:0')
    layer4_out  = graph.get_tensor_by_name('layer4_out:0')
    layer7_out  = graph.get_tensor_by_name('layer7_out:0')
        
    return input_image, keep_prob, layer3_out, layer4_out, layer7_out

tests.test_load_vgg(load_vgg, tf)


def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    # 1x1 convolutions of the three layers
    init_sd = 0.01
    layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1, kernel_initializer= tf.truncated_normal_initializer(stddev=init_sd))
    layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, 1, kernel_initializer=tf.truncated_normal_initializer(stddev=init_sd))
    layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1, kernel_initializer=tf.truncated_normal_initializer(stddev=init_sd))

    # Upsample layer 7 and add to layer 4
    total = tf.layers.conv2d_transpose(layer7,    num_classes,  4, 2, 
                                       'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=init_sd))
    total = tf.add(total, layer4)

    # Upsample the sum and add to layer 3
    total = tf.layers.conv2d_transpose(total, num_classes,  4, 2, 
                                       'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=init_sd))
    total = tf.add(total, layer3)

    # Upsample the total and return
    total = tf.layers.conv2d_transpose(total, num_classes, 16, 8, 
                                       'SAME', kernel_initializer=tf.truncated_normal_initializer(stddev=init_sd))
    return total
    
tests.test_layers(layers)


def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    
    # Reshape logits and label for computing cross entropy
    logits        = tf.reshape(nn_last_layer, (-1, num_classes), name='logits')
    correct_label = tf.reshape(correct_label, (-1, num_classes))

    # Compute cross entropy and loss
    cross_entropy_logits = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label)
    cross_entropy_loss   = tf.reduce_mean(cross_entropy_logits)

    # Define a training operation using the Adam optimizer
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)
    return logits, train_op, cross_entropy_loss

tests.test_optimize(optimize)


def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    training_images = 289
    dropout = 0.5
    llearning_rate = 0.00001
    # Iterate over epochs
    for epoch in range(1, epochs+1):
        print("Epoch: " + str(epoch) + "/" + str(epochs))

        # Iterate over the batches using the batch generation function
        total_loss = []
        batch = get_batches_fn(batch_size)
        size = math.ceil(training_images / batch_size)

        for i, d in tqdm(enumerate(batch), desc="Batch", total=size):

            # Create the feed dictionary
            image, label = d
            feed_dict = { 
                input_image   : image,
                correct_label : label,
                keep_prob     : dropout,
                learning_rate : llearning_rate
            }

            # Train and compute the loss
            _, loss = sess.run([train_op, cross_entropy_loss], feed_dict=feed_dict)
            total_loss.append(loss)

            # Compute mean epoch loss
            mean_loss = sum(total_loss) / size
            print("Loss:  " + str(loss) + "\n")

    
tests.test_train_nn(train_nn)

'''
  Save the model
'''
def save_model(sess):
    saver = tf.train.Saver()
    saver.save(sess, save_location + 'variables/saved_model')
    tf.train.write_graph(sess.graph_def, save_location, "saved_model.pb", False)

def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        
        # Placeholders
        learning_rate = tf.placeholder(dtype = tf.float32)
        correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, num_classes))
        
        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        
        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3, layer4, layer7   = load_vgg(sess, vgg_path)
        output = layers(layer3, layer4, layer7, num_classes)
        logits, train_op, cross_entropy_loss = optimize(output, correct_label, learning_rate, num_classes)
        
        # Initialize variables 
        sess.run(tf.global_variables_initializer())
            
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, 
                     cross_entropy_loss, input_image, correct_label, keep_prob, learning_rate)
        
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
                
        # Save the model
        save_model(sess)

        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    
    #Global parameters definition
    learning_rate = 0.00001
    dropout = 0.5
    epochs = 100
    batch_size = 4
    init_sd = 0.01
    training_images = 289
    num_classes = 2
    image_shape = (160, 576)
    data_dir = 'data'
    runs_dir = 'runs'
    training_subdir = 'data_road/training'
    save_location = 'data/fcn/'
    
    #Run the program
    run()
